In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
# Set default plotly theme
pio.templates.default = 'plotly_white'

from src.utilities import Matrix

from src.observables import *

SUFFIX = '.npy'
OMEGA = 1  # Interaction Strength

HEAT = False

### Functions

In [ ]:
def single_entropy(cov, system):
    d1, d2 = symplectic_eigenvalues(cov)
    if system == 1:
        x1 = d1 + 1/2
        x2 = d1 - 1/2
    elif system == 2:
        x1 = d2 + 1/2
        x2 = d2 - 1/2
    return x1 * np.log(x1) - x2 * np.log(x2)

In [ ]:
def get_observable(observable, covs, iterator):
    if observable == 'Entropy':
        func = symplectic_entropy
    elif observable == 'Mutual Information':
        func = mutual_information
    elif observable == 'Quantum Discord':
        func = gaussian_quantum_discord
    elif observable == 'Logarithmic Negativity':
        func = logarithmic_negativity
    else:
        raise ValueError(f'Observable {observable} not supported')

    arr = []
    for i in iterator:
        arr.append(func(covs[i]))
    return np.array(arr).real

# Effects of Interaction Time on the evolution
For the paper we inspect:
1. the difference in the evolution of the observables for different interaction times.
2. the difference of the partial evolution of observables for different interaction times. 

For (1) use data with IDs [032, 033, 034, 037] and for (2) use data with IDs [035, 036]. 

In [ ]:
log = pd.read_csv('../objects/saved/logs.csv', sep=',', decimal='.', index_col='Id')
# Format the index values as strings with leading zeros
log.index = log.index.astype(str).str.zfill(3)

# Filter the data to load
filter_ids = ['01S', '02S', '032']

# Filter the DataFrame using the query method and the filtered IDs
files_to_load = log.loc[filter_ids]
print(f"There are {len(files_to_load)} files to load")

In [ ]:
observables = ['Temperatures', 'Entropies', 'Mutual Information', 'Quantum Discord', 'Logarithmic Negativity', 'Heat Flux']
df = pd.DataFrame(columns=observables)
steps_per_timedelta = 1

steps = 10000  # To filter data points

print(f'Loading files with:\n'
      f'{"Evolution Steps":>21}\t'
      f'{"Log ID":>10}\t'
      f'{"Interaction Time":>21}\t'
      f'{"Interaction Strength":>21}\t')
for indx, metadata in files_to_load.iterrows():
    dt_df = pd.DataFrame(columns=observables)
    timedelta = metadata['timedelta']
    omega = metadata['omega']
    
    cov_filename = f'../objects/saved/{indx}_rho_covariance.npy'
    heat_filename = f'../objects/saved/{indx}_rho_heats.npy'
    cov_evolution = np.load(cov_filename)
    heat_transfer = np.load(heat_filename).real if HEAT else np.zeros((len(cov_evolution),3))
    
    print(f'{len(cov_evolution):>21}\t'
          f'{indx:>10}\t'
          f'{timedelta:>21}\t'
          f'{omega:>21}')
    
    # Iterator to select only a sample of data
    iterator = np.linspace(0, len(cov_evolution) - 1, steps).astype(int)
    heat_iterator = np.linspace(0, len(heat_transfer) - 1, steps).astype(int)
    
    dt_df['Log ID'] = [indx for _ in iterator]
    dt_df['Time'] = [i * timedelta for i in iterator]
    dt_df['Interaction Time'] = timedelta
    dt_df['d1'] = np.array([symplectic_eigenvalues(cov_evolution[i])[1] for i in iterator]).real
    dt_df['d2'] = np.array([symplectic_eigenvalues(cov_evolution[i])[1] for i in iterator]).real
    dt_df['d1-'] = np.array([symplectic_eigenvalues_transposed(cov_evolution[i])[1] for i in iterator]).real
    dt_df['d2-'] = np.array([symplectic_eigenvalues_transposed(cov_evolution[i])[1] for i in iterator]).real

    for obs in observables:
        if obs == 'Heat Flux':
            dt_df['J1'] = np.array([heat_transfer[i, 0] for i in heat_iterator])
            dt_df['J2'] = np.array([heat_transfer[i, 1] for i in heat_iterator])
            dt_df['Jc'] = np.array([heat_transfer[i, 2] for i in heat_iterator])
        elif obs == 'Entropies':
            dt_df['S'] = np.array([symplectic_entropy(cov_evolution[i]) for i in iterator]).real
            dt_df['S1'] = np.array([single_entropy(cov_evolution[i], 2) for i in iterator]).real
            dt_df['S2'] = np.array([single_entropy(cov_evolution[i], 1) for i in iterator]).real
        elif obs == 'Temperatures':
            dt_df['N1'] = np.array([mean_photon_numbers(cov_evolution[i])[0] for i in iterator]).real
            dt_df['N2'] = np.array([mean_photon_numbers(cov_evolution[i])[1] for i in iterator]).real
        else:
            dt_df[obs] = get_observable(obs, cov_evolution, iterator)

    df = pd.concat([df, dt_df])
df.drop('Heat Flux', axis=1, inplace=True)
df.drop('Entropies', axis=1, inplace=True)
df.drop('Temperatures', axis=1, inplace=True)

In [ ]:
fig_n, ax = plt.subplots()
ls_dict = {
    1.0: '-',
    0.5: '--',
    0.1: '-.',
    0.7: ':',
    0.65: '-',
    0.6: '--',
    0.45: '-',
    0.55: '--',
}
for indx, group in df.groupby('Log ID'):
    timedelta = log.loc[indx, 'timedelta']
    label = f"{indx} + {tuple(log.loc[indx, ['omega', 'timedelta']])}"
    ax.plot(group['Time'], group['S'],ls=':', label=label)
ax.legend()
ax.set_xlim(0, 1000)

In [ ]:
df_melt = df.melt(id_vars=['Time', 'Log ID'], value_vars=['N1', 'N2'], var_name='System', value_name='Temperature')
fig_temp = px.line(df_melt, x='Time', y='Temperature',
                   color='Log ID', line_dash='System',
                   title='Mean Photon Numbers in the Systems',
                   labels={'Temperature': 'Mean Photon '
                                          'Number', 'Time': 'Time (a.u.)'})
fig_temp.update_xaxes(range=[0, 1500])
fig_temp.show()

In [ ]:
df_melt = df.melt(id_vars=['Time', 'Log ID'], value_vars=['S1', 'S2'], var_name='System', value_name='Entropy')
fig_entropy = px.line(df_melt, x='Time', y='Entropy',
                   color='Log ID', line_dash='System',
                   title='Systems Entropies')
fig_entropy.update_xaxes(range=[0, 1500])
fig_entropy.show()

In [ ]:
fig_discord = px.line(df, x='Time', y='Quantum Discord', color='Log ID', title='Quantum Discord')
fig_discord.update_xaxes(range=[0, 200])
fig_discord.show()

In [ ]:
fig_mutinf = px.line(df, x='Time', y='Mutual Information', color='Log ID', title='Mutual Information')
fig_mutinf.update_xaxes(range=[0, 1500])
fig_mutinf.show()

In [ ]:
fig_logneg = px.line(df, x='Time', y='Logarithmic Negativity', color='Log ID', title='Logarithmic Negativity')
fig_logneg.update_xaxes(range=[0, 1500])
fig_logneg.show()

In [ ]:
df_melt = df.loc[df['Time']>0].melt(id_vars=['Time', 'Interaction Time'], value_vars=['J1', 'J2', 'Jc'], var_name='Current', value_name='Heat')
fig_heat = px.line(df_melt, x='Time', y='Heat',
                   color='Interaction Time', line_dash='Current',
                   title='Heat Currents')
fig_heat.update_xaxes(range=[0, 200])
fig_heat.update_yaxes(range=[-0.005, 0.0001])
fig_heat.show()

## Symplectic Eigenvalues

In [ ]:
fig, ax = plt.subplots()
x=df.loc[df['Interaction Time'] == 1.0]['Time']
y1=df.loc[df['Interaction Time'] == 1.0]['d1']
y2=df.loc[df['Interaction Time'] == 1.0]['d1-']
ax.plot(x, y1, y2)
plt.show()

In [ ]:
fig, ax = plt.subplots()

x=df.loc[df['Interaction Time'] == 1.0]['Time']
y1=df.loc[df['Interaction Time'] == 1.0]['d2']
y2=df.loc[df['Interaction Time'] == 1.0]['d2-']

ax.plot(x, y1, y2)
plt.show()

## Save Data Frame

In [ ]:
pd.to_pickle(df, f'../objects/saved/observables.pkl')